In [1]:
import sys, os, json
import pandas as pd
import numpy as np
import lightgbm as lgb
from lightgbm import LGBMClassifier
import pickle
from tqdm import tqdm
from smart_open import open

import gc
import time 
import warnings
import random
import ast

warnings.simplefilter(action='ignore')
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 500)

%load_ext autoreload
%autoreload 2
print(lgb.__version__)

3.2.1


In [2]:
config_path = "./config.json"

with open(os.path.join(config_path), "r") as f:
    config = json.load(f)

#### dev1
- sample subset (10%, ~1000 bad on booked)
- encode special to missing

In [3]:
exp_dict = pd.read_csv(config["meta"]["exp_dict_path"])

In [18]:
config["data_columns"].keys()

dict_keys(['bureau_features_cols', 'customized_cols', 'af_member_cols', 'cat_cols', 'meta_cols', 'target_cols', 'weights', 'benchmark_cols', 'decision_cols'])

In [26]:
ratio = 0.1
cols_ = config["data_columns"]["bureau_features_cols"][:100] + config["data_columns"]["meta_cols"] + config["data_columns"]["target_cols"] + config["data_columns"]["weights"]


In [27]:
df_dev1 = pd.read_parquet(config["data"]["clean"]["all_features_dev1"], columns=cols_)

In [164]:
ratio = 0.1
random.seed(12345)
source = ['booked','proxy','others']
df_dev1_sub = pd.DataFrame()
for i in tqdm(source):
    ids = random.sample(df_dev1[df_dev1.ri_source==i].id.unique().tolist(), 
                                      round(df_dev1[df_dev1.ri_source==i].id.unique().size * ratio))
    df_i = df_dev1[df_dev1.id.isin(ids)].copy()
    df_dev1_sub = pd.concat([df_dev1_sub, df_i], axis=0)

100%|██████████| 3/3 [00:03<00:00,  1.32s/it]


In [167]:
df_dev1.shape, df_dev1_sub.shape

((2281891, 112), (228188, 112))

In [184]:
df_dev1 = pd.read_parquet(config["data"]["clean"]["all_features_dev1"])

In [33]:
# def categorical_to_nan(series, categorical_list):
#     """ given series and a list of catergorical values
    
#     replace the categorical occurances to nan
#     """
#     if len(categorical_list) == 0:
#         return series
#     mapper = dict((k, np.nan) for k in categorical_list)
#     return series.replace(mapper)

# no_special_list = []
# list_features = config["data_columns"]["bureau_features_cols"] 

# for col in tqdm(list_features):
#     try:
#         special_val = ast.literal_eval(exp_dict[exp_dict.field_name == col].categorical.iloc[0])
#         special_val = [int(i) for i in special_val]
#         df_dev1_sub[col] = categorical_to_nan(df_dev1_sub[col], special_val)
#     except:
#         no_special_list.append(col)

In [49]:
exp_dict.field_name

0                     base_ts
1       experian_consumer_key
2                 p13_alj0300
3                 p13_alj0313
4                 p13_alj0316
                ...          
4814             t11_tupl3421
4815             t11_tupl3422
4816             t11_tupl4420
4817             t11_tupl4421
4818             t11_tupl4422
Name: field_name, Length: 4819, dtype: object

In [59]:
col = "t11_tupl3422"
exp_dict[exp_dict.field_name == col].categorical.item()

'[999, 998, 997, 996]'

In [67]:
exp_dict.categorical.value_counts()

[999999999, 999999998, 999999997, 999999996]    1641
[99, 98, 97, 96]                                 405
[99, 98, 97]                                     317
[999, 998, 997, 996]                             308
[99, 98]                                         296
                                                ... 
[999, 997, 996, 995, 994]                          1
[999, 996, 995]                                    1
[999, 997, 996, 994]                               1
[999, 998, 996]                                    1
[9999, 9998, 9996, 9995]                           1
Name: categorical, Length: 69, dtype: int64

In [122]:
def parse_categorical(item):
    try:
        return ast.literal_eval(item)
    except:
        if np.isnan(item):
            return []
        raise ValueError()
        
        
# def categorical_to_nan(series, categorical_list):
#     """ given series and a list of catergorical values
    
#     replace the categorical occurances to nan
#     """
#     if len(categorical_list) == 0:
#         return series
#     mapper = dict((k, np.nan) for k in categorical_list)
#     return series.replace(mapper)


def categorical_to_nan(df, col):
    """ given series and a list of catergorical values
    
    replace the categorical occurances to nan
    """
    categorical_list = dict_special_val[col]
    if len(categorical_list):
        return df[col]
    mapper = dict((k, np.nan) for k in categorical_list)
    return df[col].replace(mapper)

In [118]:
cols = config["data_columns"]["bureau_features_cols"][:100]
df = dd.read_parquet("df_dev1_sub.parquet", columns=cols)
results = {}
for col in cols: #list_features:
    results[col] = categorical_to_nan(df, col)
    
results = dd.compute(results)

In [186]:
%%time
import random
import pandas as pd
import numpy as np
from multiprocessing import Pool

no_special_list = []
list_features = config["data_columns"]["bureau_features_cols"] 

dict_special_val = {}
for col in list_features:
    special_val = parse_categorical(exp_dict[exp_dict.field_name == col].categorical.iloc[0])       
    special_val = [int(i) for i in special_val]
    dict_special_val[col] = special_val
    

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df


def categorical_to_nan(df):
    """ given series and a list of catergorical values
    
    replace the categorical occurances to nan
    """
    for col in list_features:
        if col not in df.columns:
            continue
        categorical_list = dict_special_val[col]
        if len(categorical_list) == []:
            continue
        mapper = dict((k, np.nan) for k in categorical_list)
        df[col] = df[col].replace(mapper)
    return df

df = df_dev1_sub.copy(deep=True)
df_ = parallelize_dataframe(df, categorical_to_nan, n_cores=32)
print(df.shape, df_.shape)

MemoryError: Unable to allocate 82.2 GiB for an array with shape (4835, 2281891) and data type int64

In [153]:
df_.head()

,p13_alj0300,p13_alj0313,p13_alj0316,p13_alj0416,p13_alj5030,p13_alj5320,p13_alj5530,p13_alj5730,p13_alj5820,p13_alj5830,...,p13_all2206,p13_all2216,p13_all2220,p13_all2226,p13_all2300,p13_all2306,p13_all2307,p13_all2308,p13_all2309,p13_all2320
103,2.0,1.0,1.0,1.0,178922.0,190400.0,178922.0,0.0,861.0,861.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
154,3.0,1.0,1.0,1.0,12973.0,13825.0,12973.0,0.0,480.0,480.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
656,11.0,5.0,6.0,5.0,423573.0,478571.0,84715.0,0.0,5186.0,5186.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
929,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0,2.0,3.0


In [152]:
df__.head()

,p13_alj0300,p13_alj0313,p13_alj0316,p13_alj0416,p13_alj5030,p13_alj5320,p13_alj5530,p13_alj5730,p13_alj5820,p13_alj5830,...,p13_all2206,p13_all2216,p13_all2220,p13_all2226,p13_all2300,p13_all2306,p13_all2307,p13_all2308,p13_all2309,p13_all2320
103,2,1,1,1,178922,190400,178922,0,861,861,...,0,0,0,0,0,0,0,0,0,0
148,3,0,0,0,999999997,999999997,999999997,999999997,999999997,999999997,...,0,0,0,0,0,0,0,0,0,0
154,3,1,1,1,12973,13825,12973,0,480,480,...,0,0,0,0,1,0,0,0,0,3
656,11,5,6,5,423573,478571,84715,0,5186,5186,...,0,0,0,0,0,0,0,0,0,0
929,0,98,98,98,999999998,999999998,999999998,999999998,999999998,999999998,...,0,0,0,0,3,1,1,1,2,3


In [ ]:
class Cat2Nan(object):
    
    def __init__(self, )

In [187]:
%%time
def categorical_to_nan(series, categorical_list):
    """ given series and a list of catergorical values
    
    replace the categorical occurances to nan
    """
    if len(categorical_list) == 0:
        return series
    mapper = dict((k, np.nan) for k in categorical_list)
    return series.replace(mapper)

no_special_list = []
list_features = config["data_columns"]["bureau_features_cols"] 

df = df_dev1_sub.copy(deep=True)
for col in tqdm(list_features):
    try:
        special_val = ast.literal_eval(exp_dict[exp_dict.field_name == col].categorical.iloc[0])
        special_val = [int(i) for i in special_val]
        df[col] = categorical_to_nan(df_dev1_sub[col], special_val)
    except:
        no_special_list.append(col)

MemoryError: Unable to allocate 82.2 GiB for an array with shape (4835, 2281891) and data type int64

In [188]:
df_.shape, df.shape

((2281891, 112), (2281891, 112))

#### dev2
- sample subset (10%, ~1000 bad on booked)
- encode special to missing

In [ ]:
df_dev2 = pd.read_parquet(config["data"]["clean"]["all_features_dev2"])

In [ ]:
ratio = 0.1
random.seed(12345)
source = ['booked','proxy','others']
df_dev2_sub = pd.DataFrame()
for i in tqdm(source):
    ids = random.sample(df_dev2[df_dev2.ri_source==i].id.unique().tolist(), 
                                      round(df_dev2[df_dev2.ri_source==i].id.unique().size * ratio))
    df_i = df_dev2[df_dev2.id.isin(ids)].copy()
    df_dev2_sub = pd.concat([df_dev2_sub, df_i], axis=0)
    
print(df_dev2.shape)
print(df_dev2_sub.shape)

In [ ]:
no_special_list = []
list_features = config["data_columns"]["bureau_features_cols"] 

for col in tqdm(list_features):
    try:
        special_val = ast.literal_eval(exp_dict[exp_dict.field_name == col].categorical.iloc[0])
        special_val = [int(i) for i in special_val]
        df_dev2_sub[col] = categorical_to_nan(df_dev2_sub[col], special_val)
    except:
        no_special_list.append(col)

In [ ]:
# df_dev2_sub.to_parquet("s3://sofi-data-science/hpeng/pl-gen4/data_dump/cleaned_tables/uw/all_features_dev2_sample_encoding_v1.parquet")